In [21]:
import warnings
import itertools
import pickle
import copy
import math
import bisect
import torch
import json
import numpy as np
import pandas as pd
import torch.nn as nn
from transformers import BertModel
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


F_P = {
    0: [0,1],
    1: [2,3],
    2: [4,5,6,7,8,9,10,11],
    3: [12,13],
}
    

def generate_partition(f_part):  #input a dict
    setnum = len(f_part)  #子集數
    num_set = [i for i in range(setnum)]  #子集數list
    cardlist = [len(i) for i in f_part.values()]  #各子集cardinality
    f_list = [i for i in f_part.values()]  #list of list
    all_subsets = []
    for r in range(setnum+1):
        for s in combinations(num_set, r):
            tmp = []
            for t in s:
                tmp.extend(f_list[t])
            all_subsets.append(np.array(sorted(tmp)))
    # Create a hash table to store the index of each subset
    subset_index_table = {}
    for index, value in enumerate(all_subsets):
        subset_index_table[tuple(value)] = index
    return all_subsets, subset_index_table, len(all_subsets)


class Hypercube_wpart:
    '''
    A class to create a hypercube object which stores values of vertices
    '''    
    #輸入維度
    def __init__(self, partition):   #input a dict
        self.f_part = partition
        self.n_dim = len(self.f_part)
        self.elem_count = sum([len(i) for i in self.f_part.values()])
        # vertex_values is a dictionary to store the value of each vertex.
        # Because np.array is not hashable, we use tuple to represent the vertex.
        self.vertex_values = {}
        self.vertices, self.vertex_index, self.vertex_num = generate_partition(self.f_part)  #vertex 上的value一次考慮整個subset
        self.edges, self.edge_num = self.build_edges()
        self.differential_matrix = None
        self.weight_matrix = None
        self.generate_min_l2_norm_matrix()
    
    def build_edges(self):
        num_set = [i for i in range(self.n_dim)]  #子集數list
        s_set = set(num_set)  #轉集合
        cardlist = [len(i) for i in self.f_part.values()]  #各子集cardinality
        f_list = [i for i in self.f_part.values()]  #list of list
        #print(f'Receive {f_list}')
        s_subset = set(tuple(i) for i in f_list)
        edges = []
        for r in range(self.n_dim): 
            for v in combinations(num_set, r):
                v_set = set(v)
                adjunct_v = s_set - v_set
                for new_elem in adjunct_v:
                    d_set = v_set | {new_elem}
                    outlist, inlist = [],[]                    
                    for k in v_set:
                        outlist.extend(f_list[k])
                    for l in d_set:
                        inlist.extend(f_list[l])
                    edges.append(((np.array(sorted(outlist))),np.array(sorted(inlist))))
        return edges, len(edges)
    
    def get_elements(self, index):
        return tuple(self.f_part[index])

    def set_vertex_values(self, vertex_values):         #設置點值
        for v in vertex_values:                         #用鍵值來做查找
            self.vertex_values[v] = vertex_values[v]
        
    def does_edge_exist(self, v1, v2):
        if abs(len(v1)-len(v2))==1:
            interset = np.intersect1d(v1,v2)
            smaller = v1 if len(v1)<len(v2) else v2
            return True if np.array_equal(smaller, interset) else False
        else:
            return False
    
    # Establish the matrix A in the above formula: AX-Y
    def generate_differential_matrix(self):
        if self.differential_matrix is None:
            self.differential_matrix = np.zeros((self.edge_num+1, self.vertex_num))
            for i,v_pair in enumerate(self.edges):
                j = self.vertex_index[tuple(v_pair[1])]
                k = self.vertex_index[tuple(v_pair[0])]
                self.differential_matrix[i][j] = 1
                self.differential_matrix[i][k] = -1
            # Add one more equestion that x_0 = 0 into the matrix form
            self.differential_matrix[-1][0]=1
        return self.differential_matrix

    # Pre-calcuate "W=(A^T*A)^-1*A^T" for the formula "X = ((A^T*A)^-1*A^T)*Y
    def generate_min_l2_norm_matrix(self):
        matrix_A = self.generate_differential_matrix()
        matrix_A_T = np.transpose(matrix_A)
        self.weight_matrix = np.linalg.inv(matrix_A_T @ matrix_A) @ matrix_A_T

    def get_gradient_vector(self):
        gradient_vector = np.zeros(self.edge_num)
        for i,v_pair in enumerate(self.edges):
            gradient_vector[i] = self.vertex_values[tuple(v_pair[1])]-self.vertex_values[tuple(v_pair[0])]    
        return gradient_vector      
        
    def get_partial_gradient_vector(self,subset_i):  #feature->subset->allow input int or tuple
        if isinstance(subset_i, int):
            feature_i = self.get_elements(subset_i)
        else:
            feature_i = []            
            for i in subset_i:
                feature_i.append(self.get_elements(i))
            feature_i = tuple(chain.from_iterable(feature_i))
        partial_gradient_vector = np.zeros(self.edge_num)
        for i,v_pair in enumerate(self.edges):
            if (not set(feature_i).issubset(set(v_pair[0]))) and (set(feature_i).issubset(set(v_pair[1]))):
                partial_gradient_vector[i] = self.vertex_values[tuple(v_pair[1])]-self.vertex_values[tuple(v_pair[0])]    
        return partial_gradient_vector
    
    def resolve_vi(self, subset_i, phi_0=0):  #feature->subset
        pd = self.get_partial_gradient_vector(subset_i)
        # Append equation x_0=0 at the end of partial gradient vector.
        pd = np.append(pd, phi_0)
        vi = self.weight_matrix @ pd
        # Reconstruct the vertex values
        new_vertices = {}
        for i,v in enumerate(self.vertices):
            new_vertices[tuple(v)] = vi[i]
        return vi, new_vertices




In [12]:
import pandas as pd
df_10 = pd.read_csv('10instance.csv',index_col='index')
df_10 = df_10.drop(columns=['Output','DFT'])
background = pd.read_csv('background_dataset_2.csv',index_col = 'Unnamed: 0')
background = background.drop(columns=['Output'])
df = pd.read_csv('data_with_output.csv')
df = df.drop(columns=['Output'])
print(df.head(3))

   X_-TACT_TIME_mean  X_-CONVEYOR_SPEED_mean  PUMP_high  PUMP_low  \
0                 90                    3150   29848.68  10349.70   
1                 90                    4200   37183.04  12891.48   
2                 90                    4200   40499.96  12198.94   

   CLN1_over-etching-ratio  CLN1_EPT_time  clean_count  EPT_clean_count_ratio  \
0                 0.001802           9987            3                 3329.0   
1                 0.003292          10023            2                 5011.5   
2                 0.005368          10059            3                 3353.0   

   NH3_TREAT_-RF_FREQ-max  NH3_TREAT_-RF_FREQ-range  NH3_TREAT_-RF_FREQ-mean  \
0                   13947                       444               13614.0000   
1                   13963                       442               13623.0000   
2                   13948                       418               13649.4286   

   NP_3_-MFC_VOL_SIH4-range    VENT_high     VENT_low  
0                    

In [13]:
mean1 = df.mean()
std1 = df.std()
print(mean1)
print(std1)

X_-TACT_TIME_mean             154.203192
X_-CONVEYOR_SPEED_mean       3132.278651
PUMP_high                   42443.758562
PUMP_low                    11360.190378
CLN1_over-etching-ratio         0.004456
CLN1_EPT_time               11845.178373
clean_count                     3.684303
EPT_clean_count_ratio        4821.724736
NH3_TREAT_-RF_FREQ-max      13901.145963
NH3_TREAT_-RF_FREQ-range      383.961172
NH3_TREAT_-RF_FREQ-mean     13623.735749
NP_3_-MFC_VOL_SIH4-range        1.005759
VENT_high                   16660.791956
VENT_low                     7010.155928
dtype: float64
X_-TACT_TIME_mean              14.183177
X_-CONVEYOR_SPEED_mean        352.664542
PUMP_high                   12038.304024
PUMP_low                     1858.782605
CLN1_over-etching-ratio         0.004441
CLN1_EPT_time                1218.368666
clean_count                     2.017230
EPT_clean_count_ratio        3528.616445
NH3_TREAT_-RF_FREQ-max        146.644850
NH3_TREAT_-RF_FREQ-range      138.132554
N

In [14]:
c,_,_ = generate_partition(F_P)
collist = [i.tolist() for i in c]
colname = [tuple(i) for i in collist]
vertex_df = pd.DataFrame(columns=colname)
print(vertex_df.shape,vertex_df)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
s1_model_path = 'stage_1_checkpoint.pth'
s1_model =  torch.load(s1_model_path, map_location=device).to(device)
json_file_path = 'controllable_para_v014_14.json'
tool_name = 'ASCVD'
with open(json_file_path, 'r') as f:
    params = json.load(f)[tool_name]
    print(params)
    f.close()
target_features = []

''' 
    The feature_translation is a list of tuples, each tuple contains two integers.
    The tuples record the correponding position of the ith feature in the 4*freature matrix
    as the input the prediction model.
    Ex.
    [(0, 0), (0, 1), 
     (1, 2), (1, 3), 
     (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), 
     (3, 12),(3, 13)]
'''
# Flat the feature list and construct teh freature translation list to map feature into the input of the model
feature_translation = []
sub_op_num = 0
for entry in [sub_op for _, sub_op in params.items()]:
    if isinstance(entry, str):
        target_features.append(entry)
    else:
        target_features.extend(entry)
        feature_translation.extend([(sub_op_num,len(feature_translation)+j) for j in range(len(entry))])
        sub_op_num = sub_op_num+1 

print(target_features)
print(f"There are {sub_op_num} sub processes. So the feature translation is {feature_translation}")

def padding_zero(df, flag): 
    # 將一維參數matrix擴展為4維
    data_arr = df.to_numpy()
    result = []
    for data in data_arr:
        empty_arr = np.zeros((sub_op_num, len(feature_translation))) # chamber數 * 總參數數量
        for i, pos in enumerate(feature_translation):
            empty_arr[pos[0]][pos[1]] = data[i]
        if(flag == 1): # bert.py使用
            result.append(empty_arr)
        if(flag == 2): # bert_du.py使用
            result.append(empty_arr.tolist())
    
    if(flag == 1): # bert.py使用
        result = pd.DataFrame({'X': [result[i] for i in range(len(result))]})
    return result

print(device)

(0, 16) Empty DataFrame
Columns: [(), (0, 1), (2, 3), (4, 5, 6, 7, 8, 9, 10, 11), (12, 13), (0, 1, 2, 3), (0, 1, 4, 5, 6, 7, 8, 9, 10, 11), (0, 1, 12, 13), (2, 3, 4, 5, 6, 7, 8, 9, 10, 11), (2, 3, 12, 13), (4, 5, 6, 7, 8, 9, 10, 11, 12, 13), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), (0, 1, 2, 3, 12, 13), (0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)]
Index: []
{'EQ': ['X_-TACT_TIME_mean', 'X_-CONVEYOR_SPEED_mean'], 'PUMP': ['PUMP_high', 'PUMP_low'], 'CH': ['CLN1_over-etching-ratio', 'CLN1_EPT_time', 'clean_count', 'EPT_clean_count_ratio', 'NH3_TREAT_-RF_FREQ-max', 'NH3_TREAT_-RF_FREQ-range', 'NH3_TREAT_-RF_FREQ-mean', 'NP_3_-MFC_VOL_SIH4-range'], 'VENT': ['VENT_high', 'VENT_low'], 'y': 'Output'}
['X_-TACT_TIME_mean', 'X_-CONVEYOR_SPEED_mean', 'PUMP_high', 'PUMP_low', 'CLN1_over-etching-ratio', 'CLN1_EPT_time', 'clean_count', 'EPT_clean_count_ratio', 'NH3_TREAT_-RF_FREQ-max', 'NH3_TREAT_-RF_FREQ-range', '

In [15]:
def model_inference(data):  #standardize dataframe
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    s1_model.eval()
    # scaler = StandardScaler()
    if 'Output' in data.columns:
        data = data.drop(columns=['Output'])
    # data_standardized_df = pd.DataFrame(scaler.fit_transform(data))
    #print(data.head())
    #print(f'Standardized data:\n{data_standardized_df.head(3)}')  # Debug
    
    data_4d = padding_zero(data,flag=1)
    
    #print(f'4D data:\n{data_4d.head()}')  # Debug
    
    data_4d_array = np.array([e for entry in data_4d.values for e in entry])
    data_4d_tensor = torch.tensor(data_4d_array,dtype=torch.float)
    
    #print(f'Tensor data shape: {data_4d_tensor.shape}')  # Debug 
    
    my_dataset = TensorDataset(data_4d_tensor)
    batch_size = min(256, int(data_4d_tensor.size()[0]))
    my_loader = DataLoader(my_dataset, batch_size=batch_size,num_workers=4)
    data_output = []
    with torch.no_grad():
        for batch_data in my_loader:
        # 将数据移到指定的设备上（如 CUDA 设备）
            batch_data = batch_data[0].to(device)
        # 将数据传递给模型进行推理
            batch_output = s1_model(batch_data)
            probs = (torch.nn.functional.softmax(batch_output, dim=1))
        # 将输出保存起来
            data_output += probs
    data_output_array = np.array([output.cpu().numpy()[0] for output in data_output])
    data_expectation_value = data_output_array.mean()
    return data_expectation_value, data_output_array

In [16]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [17]:
para_num = 14
subset_sum = 4
#sample_num = 1000

vertices_fl = 'E_vi_20240606.pkl'
# v_hist = vertices_fl.split('.')[0]+'_history.pkl'
# try:
#     with open(v_hist, 'rb') as f:
#         print(f'Loading form {v_hist}...')
#         v_history = pickle.load(f)
#         print(type(v_history))
# except FileNotFoundError:
#     v_history = {}

# print(f"History vertices: {v_history}")
#mean_exp = new_df['Output'].mean()
AUO_coalitions, _, _ = generate_partition(F_P) 
count_n = 0
if 'Output' in df.columns: 
    df = df.drop(columns=['Output'])

In [18]:
for i_index in range(10):
    print(f'index = {i_index}')
    instance = df_10.iloc[i_index]
    std_instance = (instance - mean1) / std1
    coalition_estimated_values = {}
    print(f' std_instance = {std_instance}')
    eval_df = background
    std_eval_df = (background - mean1) / std1
    base_mean, details = model_inference(std_eval_df)
    #print(f'Base mean: {base_mean}')
    for coalition in AUO_coalitions: 
        vi_df = std_eval_df.copy()  #用copy()才不會去更改到原始的dataframe
        if len(coalition)!=0:
            vi_df.iloc[:,coalition] = std_instance.iloc[coalition]          
        #print(f'coalition = {coalition}')
        #print(vi_df.head(3))
        exp, details = model_inference(vi_df)
        #print(f'exp = {exp}')
        coalition_estimated_values[tuple(coalition)] =  exp - base_mean
        #print(f'coalition_estimated_values[tuple(coalition)] = {coalition_estimated_values[tuple(coalition)]}')
        count_n += 1  
        if count_n % 10 == 0:
            with open(vertices_fl, 'wb') as f:
                pickle.dump(vertex_df, f)
            print(f"Saved {count_n} vertex")
    #print(f'coalition_estimated_values = {coalition_estimated_values}')
    instance_df = pd.DataFrame([coalition_estimated_values])
    vertex_df = pd.concat([vertex_df, instance_df], ignore_index=True)            
   

with open(vertices_fl, 'wb') as f:
    pickle.dump(vertex_df, f)

index = 0
 std_instance = X_-TACT_TIME_mean          -4.526714
X_-CONVEYOR_SPEED_mean      3.027584
PUMP_high                  -0.436998
PUMP_low                    0.823813
CLN1_over-etching-ratio    -0.261881
CLN1_EPT_time              -1.495589
clean_count                -0.834958
EPT_clean_count_ratio       0.053782
NH3_TREAT_-RF_FREQ-max      0.421795
NH3_TREAT_-RF_FREQ-range    0.420168
NH3_TREAT_-RF_FREQ-mean    -0.014344
NP_3_-MFC_VOL_SIH4-range   -0.057673
VENT_high                  -0.157582
VENT_low                    1.399164
dtype: float64
Saved 10 vertex
index = 1
 std_instance = X_-TACT_TIME_mean          -4.526714
X_-CONVEYOR_SPEED_mean      3.027584
PUMP_high                  -0.161468
PUMP_low                    0.451236
CLN1_over-etching-ratio     0.205509
CLN1_EPT_time              -1.466041
clean_count                -0.339229
EPT_clean_count_ratio      -0.416232
NH3_TREAT_-RF_FREQ-max      0.319507
NH3_TREAT_-RF_FREQ-range    0.246421
NH3_TREAT_-RF_FREQ-mean     0

In [19]:
vertex_df

,(),"(0, 1)","(2, 3)","(4, 5, 6, 7, 8, 9, 10, 11)","(12, 13)","(0, 1, 2, 3)","(0, 1, 4, 5, 6, 7, 8, 9, 10, 11)","(0, 1, 12, 13)","(2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","(2, 3, 12, 13)","(4, 5, 6, 7, 8, 9, 10, 11, 12, 13)","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)","(0, 1, 2, 3, 12, 13)","(0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)","(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)","(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)"
0,0.0,0.155535,-0.010374,-0.085595,-0.024421,0.135363,0.056374,0.161163,-0.097013,-0.041486,-0.163453,0.011295,0.149928,0.092972,-0.194775,-0.003234
1,0.0,0.155535,-0.009950,-0.054616,0.020290,0.141272,0.015947,0.155482,-0.063358,0.009653,-0.026907,-0.049967,0.128872,0.027994,-0.031373,-0.099604
2,0.0,0.155535,-0.000844,0.011546,0.000894,0.133285,0.119925,0.139673,0.035556,-0.000988,0.019490,0.106507,0.108694,0.086382,0.047812,0.063568
3,0.0,0.155535,0.003487,-0.110151,-0.049413,0.159779,0.140833,0.145138,-0.138458,-0.053916,-0.109430,0.160772,0.161854,0.138468,-0.139686,0.183511
4,0.0,0.155535,-0.029717,-0.023518,-0.018372,0.155713,0.085068,0.158429,-0.063635,-0.044645,-0.028191,0.095306,0.150578,0.098137,-0.069919,0.103981
5,0.0,0.155535,-0.014853,-0.329010,0.014576,0.136593,-0.311704,0.156713,-0.335085,-0.001809,-0.327938,-0.327367,0.135295,-0.315931,-0.329839,-0.325756
6,0.0,0.155535,-0.020856,-0.038206,0.017256,0.141932,0.150388,0.148470,-0.107440,-0.001680,-0.043927,0.125667,0.126153,0.145188,-0.096951,0.093205
7,0.0,0.155535,0.026192,-0.057218,0.023124,0.137265,0.171963,0.164149,-0.022826,0.056437,-0.033531,0.126318,0.155541,0.171570,0.023015,0.166818
8,0.0,0.155535,-0.006117,-0.310767,0.019851,0.136254,-0.304989,0.161524,-0.331208,0.013685,-0.305932,-0.321080,0.136146,-0.306904,-0.325371,-0.322777
9,0.0,0.155535,0.018429,-0.054660,-0.006771,0.135212,0.066106,0.183676,0.032272,0.017634,-0.021572,0.104844,0.181464,0.141849,0.038138,0.156815


In [22]:
#list of Hypercube_wpart and set vertex values
subsets, _, _ = generate_partition(F_P)
cubelist = []
for idx in range(vertex_df.shape[0]):
    tp = Hypercube_wpart(F_P)
    vertices = {}
    values = vertex_df.iloc[idx].tolist()
    for v in subsets:
        vertices[tuple(v)] = values.pop(0)
        tp.set_vertex_values(vertices)
    cubelist.append(tp)

#get partial gradient vector and residual for n*subsets
rows, cols = len(vertex_df), len(F_P)
pg_m = [[None for i in range(cols)] for j in range(rows)]  #partial gradient vector matrix in 3D
residual_m = [[None for i in range(cols)] for j in range(rows)]  #residual matrix in 3D
for i in range(rows):
    for j in range(cols):
        pd = cubelist[i].get_partial_gradient_vector(j)  #partial gradient of instance i of subset j 
        pg_m[i][j] = pd
        vi, new_vs = cubelist[i].resolve_vi(j)
        h1 = Hypercube_wpart(F_P)
        h1.set_vertex_values(new_vs)
        dvi = h1.get_gradient_vector()
        residual_m[i][j] = dvi-pd

#l2 norm for single subset
import pandas as pd
ptgd_l2 = [[float(np.linalg.norm(pg_m[i][j])) for j in range(cols)] for i in range(rows)]
r_l2 = [[float(np.linalg.norm(residual_m[i][j])) for j in range(cols)] for i in range(rows)]
scaled_norm = [[float(r_l2[i][j] / ptgd_l2[i][j]) for j in range(cols)] for i in range(rows)]
scn_df = pd.DataFrame(scaled_norm)
ptgd_df = pd.DataFrame(ptgd_l2)
r_df = pd.DataFrame(r_l2)
l2_mean = scn_df.mean()
l2_mean

0    0.268390
1    0.525856
2    0.394537
3    0.610870
dtype: float64

In [23]:
scn_df

,0,1,2,3
0,0.173663,0.501347,0.201707,0.639218
1,0.488389,0.574892,0.377206,0.726321
2,0.282542,0.695426,0.704973,0.613251
3,0.174366,0.742145,0.533252,0.662889
4,0.121902,0.554081,0.314283,0.681894
5,0.465005,0.308166,0.119854,0.622048
6,0.134287,0.360786,0.439797,0.731782
7,0.182545,0.689851,0.579527,0.379866
8,0.486873,0.288143,0.121484,0.563165
9,0.174327,0.543718,0.553286,0.488264


In [24]:
import torch

def test_cuda():
    # Check if CUDA is available
    if torch.cuda.is_available():
        print("CUDA is available.")
        
        # Print CUDA device name
        device_name = torch.cuda.get_device_name(0)
        print(f"CUDA Device Name: {device_name}")
        
        # Create a tensor and move it to the GPU
        tensor = torch.rand(3, 3)
        print("Original Tensor:")
        print(tensor)
        
        tensor = tensor.to('cuda')
        print("Tensor on GPU:")
        print(tensor)
        
        print("CUDA test successful!")
    else:
        print("CUDA is not available. Please check your installation.")

if __name__ == "__main__":
    test_cuda()


CUDA is available.
CUDA Device Name: NVIDIA GeForce GTX 1650
Original Tensor:
tensor([[0.5783, 0.4571, 0.6441],
        [0.3112, 0.1643, 0.4027],
        [0.4028, 0.5166, 0.1754]])
Tensor on GPU:
tensor([[0.5783, 0.4571, 0.6441],
        [0.3112, 0.1643, 0.4027],
        [0.4028, 0.5166, 0.1754]], device='cuda:0')
CUDA test successful!
